In [ ]:
# Import required packages
import time
import numpy as np
import pandas as pd
from tabulate import tabulate
import torch
from tqdm import trange
from paik.solver import Solver
from paik.settings import (
    DEFAULT_NSF,
    DEFULT_SOLVER,
)

In [ ]:
from common.config import ConfigDiversity
from common.file import save_diversity

config = ConfigDiversity()

solver_param = DEFULT_SOLVER
solver_param.workdir = config.workdir
solver = Solver(solver_param=solver_param)

num_sols = config.num_sols
num_seeds = num_sols
num_poses = config.num_poses
base_stds = config.base_stds  # start, stop, step

In [ ]:
def get_numerical_ik_sols(target_pose, num_seeds):
    seeds, _ = solver._robot.sample_joint_angles_and_poses(
        n=num_seeds, return_torch=False
    )
    numer_ik = np.empty((num_seeds, solver.n))
    for i in range(num_seeds):
        numer_ik[i] = solver.robot.inverse_kinematics_klampt(
            pose=target_pose, seed=seeds[i]
        )
    return numer_ik


_, P = solver._robot.sample_joint_angles_and_poses(n=num_poses, return_torch=False)

J_ground_truth = np.empty((num_poses, num_sols, solver.n))
for i in trange(num_poses):
    J_ground_truth[i] = get_numerical_ik_sols(P[i], num_seeds)

# Save to repeat the same experiment on NODEIK
np.save(f"{config.record_dir}/numerical_ik_sols.npy", J_ground_truth)
np.save(f"{config.record_dir}/poses.npy", P)

In [ ]:
from common.evaluate import mmd_evaluate_multiple_poses

l2_mean = np.empty((len(base_stds)))
ang_mean = np.empty((len(base_stds)))
mmd_mean = np.empty((len(base_stds)))
J_hat_paik = np.empty((len(base_stds), num_poses, num_sols, solver.n))

for i, std in enumerate(base_stds):
    solver.base_std = std
    P_expand_dim = (
        np.expand_dims(P, axis=1).repeat(num_sols, axis=1).reshape(-1, P.shape[-1])
    )

    F = solver.F[
        solver.P_knn.kneighbors(
            np.atleast_2d(P), n_neighbors=num_sols, return_distance=False
        ).flatten()
    ]
    J_hat = solver.solve_batch(P_expand_dim, F, 1)
    l2, ang = solver.evaluate_pose_error_J3d_P2d(J_hat, P_expand_dim, return_all=True)
    J_hat = J_hat.reshape(num_poses, num_sols, -1)
    J_hat_paik[i] = J_hat
    l2_mean[i] = l2.mean()
    ang_mean[i] = ang.mean()
    mmd_mean[i] = mmd_evaluate_multiple_poses(
        J_hat, J_ground_truth, num_poses=num_poses
    )
    assert not np.isnan(mmd_mean[i])

save_diversity(
    config.record_dir,
    "paik",
    J_hat_paik,
    l2_mean,
    ang_mean,
    mmd_mean,
    base_stds,
)

In [ ]:
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()
# Build IKFlowSolver and set weights
ik_solver, _ = get_ik_solver("panda__full__lp191_5.25m")
l2_flow = np.empty((len(base_stds)))
ang_flow = np.empty((len(base_stds)))
mmd_flow = np.empty((len(base_stds)))
J_hat_ikflow = np.empty((len(base_stds), num_poses, num_sols, solver.n))

for i, std in enumerate(base_stds):
    J_flow = np.array(
        [ik_solver.solve(p, n=num_sols, latent_scale=std).cpu().numpy() for p in P]
    )  # (num_sols, num_poses, n)
    J_hat_ikflow[i] = J_flow
    l2, ang = solver.evaluate_pose_error_J3d_P2d(
        J_flow.transpose(1, 0, 2), P, return_all=True
    )
    l2_flow[i] = l2.mean()
    ang_flow[i] = ang.mean()
    mmd_flow[i] = mmd_evaluate_multiple_poses(
        J_flow, J_ground_truth, num_poses=num_poses
    )
    
save_diversity(
    config.record_dir,
    "ikflow",
    J_hat_ikflow,
    l2_flow,
    ang_flow,
    mmd_flow,
    base_stds,
)

# Load mmd posture diversity and present
Note that before run the following code, you need to run
- conda activate nodeik
- python nodeik_experiments.py

to obtain `df_nodeik` data 

In [ ]:
import pandas as pd
from datetime import datetime
from common.config import ConfigDiversity
config = ConfigDiversity()

df_paik = pd.read_pickle(f"{config.record_dir}/paik_posture_mmd_std.pkl")
df_ikflow = pd.read_pickle(f"{config.record_dir}/ikflow_posture_mmd_std.pkl")
df_nodeik = pd.read_pickle(f"{config.record_dir}/nodeik_posture_mmd_std.pkl")

In [ ]:
df_l2 = pd.DataFrame(
    {
        "paik": df_paik.l2.values * 1000,
        "IKFlow": df_ikflow.l2.values * 1000,
        "NODEIK": df_nodeik.l2.values * 1000,
        "base std": df_paik.base_std.values,
    }
)

df_mmd = pd.DataFrame(
    {
        "paik": df_paik.mmd.values,
        "IKFlow": df_ikflow.mmd.values,
        "NODEIK": df_nodeik.mmd.values,
        "base std": df_paik.base_std.values,
    }
)

fontsize = 24
ax1 = df_l2.plot(x="base std", grid=True, fontsize=fontsize)
ax1.set_ylabel("L2 Error (mm)", fontsize=fontsize)
ax1.set_xlabel("Base std", fontsize=fontsize)
ax1.set_title("Position Error", fontsize=fontsize)
ax1.legend(fontsize=fontsize)

ax1 = df_mmd.plot(x="base std", grid=True, fontsize=fontsize)
ax1.set_ylabel("MMD Score", fontsize=fontsize)
ax1.set_xlabel("Base std", fontsize=fontsize)
ax1.set_title("MMD Score", fontsize=fontsize)
ax1.legend(fontsize=fontsize)